In [2]:
%load_ext autoreload
%autoreload 2
%load_ext memory_profiler
from tqdm import tqdm
import sys
sys.path.append("./ner")
from languagemodel.train import *

# Configデータの読み込み

In [ ]:
data = Dataset()
data.read_config("config/csw.4k_16k.201904121018.config")
data.build_alphabet(data.lm_dir, mode="lm")
data.word_alphabet.cut_vocab(min_count=data.lm_vocab_min_count)
data.fix_alphabet()
data.describe()
data.generate_instance('lm')
data.build_pretrain_emb()
#data.save("./reusable_results/20190411.dset")

============= load config file ==================
train_dir=/home/sekine/SubwordSequenceLabeling/Repository/Chemdner/train.bioes: <class 'str'>
dev_dir=/home/sekine/SubwordSequenceLabeling/Repository/Chemdner/valid.bioes: <class 'str'>
test_dir=/home/sekine/SubwordSequenceLabeling/Repository/Chemdner/test.bioes: <class 'str'>
model_dir=/home/sekine/SubwordSequenceLabeling/Repository/NERModel/csw4k_16k.lm200k: <class 'str'>
lm_dir=/home/sekine/SubwordSequenceLabeling/Repository/LargeCorpus/large_corpus_200k.txt: <class 'str'>
pretrain_model_dir=/home/sekine/SubwordSequenceLabeling/Repository/LanguageModel/csw4k_16k.lm200k: <class 'str'>
word_emb_dir=/home/sekine/SubwordSequenceLabeling/Repository/GloVe/gv.d50: <class 'str'>
sw_num=2: <class 'int'>
sentence_piece_dirs=[/home/sekine/SubwordSequenceLabeling/Repository/SentencePiece/sp4000.model,/home/sekine/SubwordSequenceLabeling/Repository/SentencePiece/sp16000.model]: <class 'list'>
word_emb_dim=50: <class 'int'>
char_emb_dim=30: <class

112036it [14:53, 125.38it/s]

In [2]:
data = Dataset()
data.load("./reusable_results/20190411.dset")

============ Load Dset: ./reusable_results/20190411.dset ===============


# epochの初期化

In [3]:
model = LanguageModel(data)
if data.optimizer.lower() == "sgd":
    optimizer = optim.SGD(model.parameters(), lr=data.lr, momentum=data.momentum,weight_decay=data.l2)
criterion = nn.CrossEntropyLoss().cuda()
print(model)
## start training
for idx in range(1):
    epoch_start = time.time()
    temp_start = epoch_start
    print("Epoch: %s/%s" %(idx, data.lm_epoch))
    if data.optimizer == "SGD":
        optimizer = lr_decay(optimizer, idx, data.lr_decay, data.lr)
    instance_count, loss, all_loss, right_token, whole_token = 0, 0, 0, 0, 0
    random.shuffle(data.lm_Ids)

    ## set model in train model
    model.train()
    model.zero_grad()
    best_loss = 1e+10
    batch_size = data.batch_size
    batch_id = 0
    all_loss = 0
    train_num = len(data.lm_Ids)
    total_batch = train_num//batch_size+1

[SubwordRep(
  (drop): Dropout(p=0.5)
  (embeddings): Embedding(3932, 50)
  (f_lstm): LSTM(50, 25, batch_first=True)
  (b_lstm): LSTM(50, 25, batch_first=True)
), SubwordRep(
  (drop): Dropout(p=0.5)
  (embeddings): Embedding(14196, 50)
  (f_lstm): LSTM(50, 25, batch_first=True)
  (b_lstm): LSTM(50, 25, batch_first=True)
)]
LanguageModel(
  (word_lstm): WordLSTM(
    (char_rep): CharRep(
      (char_drop): Dropout(p=0.5)
      (char_embeddings): Embedding(289, 30)
      (char_lstm): LSTM(30, 25, batch_first=True, bidirectional=True)
    )
    (drop): Dropout(p=0.5)
    (word_embedding): Embedding(28147, 50)
    (droplstm): Dropout(p=0.5)
    (lstm): LSTM(200, 100, batch_first=True, bidirectional=True)
  )
  (f_linear): Linear(in_features=100, out_features=28147, bias=True)
  (b_linear): Linear(in_features=100, out_features=28147, bias=True)
)
Epoch: 0/1
 Learning rate is set as: 0.005


# ミニバッチごとに学習開始

In [4]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

for batch_id in tqdm(range(total_batch)):
    loss = 0
    start = batch_id * batch_size
    end = (batch_id + 1) * batch_size
    if end > train_num:
        end = train_num
    instance = data.lm_Ids[start:end]

    if not instance:
        continue
    batch_word, batch_wordlen, batch_wordrecover, batch_char, batch_charlen, batch_charrecover, batch_sws, batch_swlens, batch_swrecovers, batch_swfmask, batch_swbmask, _, mask = batchify_with_label(instance, data.gpu, if_train=True, label=False)
    
    print("=" * 100)
    print(batch_word.shape)
    
    instance_count += 1
    forward_out, backward_out = model.calc_next_word_id_prob(batch_word, batch_wordlen, batch_char, batch_charlen, batch_charrecover, batch_sws, batch_swlens, batch_swrecovers, batch_swfmask, batch_swbmask)
    loss = 0
    
    f_next_word_id = torch.cat([batch_word[:, 1:], torch.zeros(batch_word.shape[0], 1).long().cuda()], dim=1)
    loss = criterion(forward_out.reshape(-1, forward_out.shape[2]), f_next_word_id.reshape(-1))
    b_next_word_id = torch.cat([torch.zeros(batch_word.shape[0], 1).long().cuda(), batch_word[:, :-1]], dim=1)
    loss += criterion(backward_out.reshape(-1, backward_out.shape[2]), b_next_word_id.reshape(-1))
    
    print(loss)
    loss.backward()
    optimizer.step()
    model.zero_grad()
    

  0%|          | 0/5000 [00:00<?, ?it/s]

torch.Size([10, 392])
tensor(20.4934, device='cuda:0', grad_fn=<ThAddBackward>)


  0%|          | 1/5000 [00:00<42:05,  1.98it/s]

torch.Size([10, 345])
tensor(20.4877, device='cuda:0', grad_fn=<ThAddBackward>)


  0%|          | 2/5000 [00:00<40:33,  2.05it/s]

...

100%|█████████▉| 4998/5000 [41:47<00:01,  1.99it/s]

torch.Size([10, 364])
tensor(10.2642, device='cuda:0', grad_fn=<ThAddBackward>)


100%|█████████▉| 4999/5000 [41:47<00:00,  1.99it/s]

torch.Size([5, 298])
tensor(11.3335, device='cuda:0', grad_fn=<ThAddBackward>)


100%|██████████| 5000/5000 [41:47<00:00,  1.99it/s]


In [65]:
torch.from np.append(batch_word[:, 1:].cpu().numpy(), np.zeros((batch_word.shape[0], 1), dtype=np.int32), axis=1)

array([[1202, 9242,    5, ...,   30,  145,    0],
       [4334,   43, 3868, ...,    0,    0,    0]])

In [64]:
f_next_word_id.reshape(-1)

tensor([1202, 9242,    5,  ...,    0,    0,    0], device='cuda:0')

In [23]:
model.word_lstm.word_embedding.weight.data.

tensor([[-0.0627,  0.1805, -0.0299,  ..., -0.0271, -0.0573, -0.2425],
        [-0.3751,  0.1860, -0.5441,  ..., -0.7387, -0.5179,  1.3458],
        [-0.3301,  0.1997,  0.5848,  ...,  0.2011, -0.1758,  0.1979],
        ...,
        [-0.5266,  0.6094, -0.0115,  ...,  0.1896, -0.2558, -0.6354],
        [ 0.0582, -0.0843,  0.3483,  ..., -0.0866, -0.0714,  0.0669],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0')

In [ ]:
!CUDA_LAUNCH_BLOCKING=1 CUDA_VISIBLE_DEVICES=2 python3.7 languagemodel/train.py --config config/csw.4k_16k.1k.config